# Looking for features

- Les thèmes associés aux discours
- L'analyse en sentiment
- La longueur des phrases et la complexité du vocabulaire utilisé.


# Importation des modules

In [26]:
###### modules pour le chargement des données depuis le XML ######
import glob
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

###### modules pour la classification ######

# modèles
from sklearn.svm import LinearSVC, SVC

# vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

# création de nos transformers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer # créer nos propres transformer

# recherche des meilleurs hyperparamètres
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# résultats
from sklearn.metrics import classification_report

# sauvegarde des modèles
from joblib import dump, load

###### modules pour la visualisation ######
import matplotlib.pyplot as plt
import pandas as pd

###### miscellaneous ######
from typing import List # typage des fonctions
import numpy as np
import re
from collections import namedtuple

##### nltk #####
from nltk.corpus import stopwords

##### multi class #####
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

# Chargement des données

In [28]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test) # y_test est vide : pas accès aux résultats

In [29]:
pattern_clean = re.compile(r"[^ \w]") # pattern à utiliser pour nettoyer les données

def clean(data): # TODO: améliorer la fonction
    global pattern_clean
    return re.sub(pattern_clean, "", data).lower()

X_train_clean = [clean(x) for x in X_train]
X_test_clean = [clean(x) for x in X_test]


In [30]:
import re
pattern = re.compile(r'\d+\t(\w+(-\w+)?)')
y_test = []
with open("./corpus/deft09_parlement_ref/deft09_parlement_ref_fr.txt",'r') as file:
    line = file.readline()
    while line:
        m= re.match(pattern,line)
        if m:
            y_test.append(m.group(1))
        else:
            y_test.append('PSE')
        line = file.readline()

In [31]:
from sklearn.preprocessing import LabelEncoder

# Créer un objet LabelBinarizer
lb = LabelEncoder()

# Convertir les étiquettes de classe en un tableau binaire
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.fit_transform(y_test)

# Analyse de sentiments

In [ ]:
# échantillon pour tester plus rapidement

X_train_sample = X_train[:100]
y_train_sample = y_train[:100]
X_test_sample = X_test[:100]

In [ ]:
# Chargez le modèle de transformer pré-entraîné
sentiment_model = pipeline("sentiment-analysis", 
                           model="nlptown/bert-base-multilingual-uncased-sentiment") # uncased parce que ne prend pas en compte la casse

In [ ]:
def sentiment_pred(posts):
    global sentiment_model
    return sentiment_model(posts, truncation=True)

In [ ]:
# Ajoutez les prédictions de sentiments à votre pipeline de classification de discours politiques

pipeline_avec_sent = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
        ('sentiments', Pipeline([
          ('stats', FunctionTransformer(sentiment_pred)),
          ('vect', DictVectorizer())
        ]))
    ])),
    ('reglog', LogisticRegression())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
pipeline_avec_sent.fit(X_train_sample, y_train_sample)

cv_scores_avec_sentiment = cross_val_score(pipeline_avec_sent, X_train_sample, y_train_sample)

In [ ]:
print(np.mean(cv_scores_avec_sentiment))
print(cv_scores_avec_sentiment)

In [ ]:
pipeline_sans_sent = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
    ])),
    ('reglog', LogisticRegression())
])

pipeline_sans_sent.fit(X_train_sample, y_train_sample)
cv_scores_sans_sent = cross_val_score(pipeline_sans_sent, X_train_sample, y_train_sample)

In [ ]:
print(np.mean(cv_scores_sans_sent))
print(cv_scores_sans_sent)

# Classification en thème

In [ ]:
theme_classifier = pipeline("zero-shot-classification",
                      model="BaptisteDoyen/camembert-base-xnli")

candidate_labels = ["sport", 
                    "immigration", 
                    "économie", 
                    "sécurité",
                    "justice",
                    "science", 
                    "féminisme", 
                    "culture", 
                    "écologie", 
                    "climat",
                    "fiscalité",
                    "animal",
                    "social",
                    "agriculture",
                    "santé",
                    "travail",
                    "racisme"]
#hypothesis_template = "Ce texte parle de {}." # on peut peut-être utiliser ça pour renvoyer un dico

def theme_predictor(texts):
  """
  Prédire les thèmes dominants des textes donnés en utilisant un modèle de type "zero-shot" : BaptisteDoyen/camembert-base-xnli
  
  Parameters:
  texts (list): une liste de chaînes de caractères contenant les textes à classifier.
  
  Returns:
  list: une liste de dictionnaires, où chaque dictionnaire contient une entrée (thème, score) pour chaque thème prédit avec son score associé.
  """
  global theme_classifier
  global candidate_labels
  
  results = []
  for text in texts:
    classification = theme_classifier(text, candidate_labels)
    labels = classification["labels"]
    scores = classification["scores"]
    results.append({label:score for label, score in zip(labels, scores)})
      
  return results

In [ ]:
# Ajoutez les prédictions de thèmes à votre pipeline de classification de discours politiques

pipeline_avec_theme = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
        ('sentiments', Pipeline([
          ('stats', FunctionTransformer(theme_predictor)),
          ('vect', DictVectorizer())
        ]))
    ])),
    ('linearsvc', LinearSVC())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
#pipeline_avec_theme.fit(X_train_sample, y_train_sample)

cv_avec_theme = cross_val_score(pipeline_avec_theme, X_train_sample, y_train_sample, verbose=10)

In [ ]:
print(np.mean(cv_avec_theme))
print(cv_avec_theme)

In [ ]:
pipeline_sans_theme = Pipeline([
    ('features', FeatureUnion([
        ('ngram_tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer())
        ])),
    ])),
    ('reglog', LogisticRegression())
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
#pipeline_sans_theme.fit(X_train_sample, y_train_sample)

cv_sans_theme = cross_val_score(pipeline_sans_theme, X_train_sample, y_train_sample, verbose=10)

In [ ]:
print(np.mean(cv_sans_theme))
print(cv_sans_theme)

# Complexité du vocabulaire

In [ ]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# return : liste de dictionnaire où chaque dictionnaire -> complexité du vocabulaire.
def calculate_vocab_complexity(post):
    liste_finale = []
    # Créer un set pour stocker les mots uniques
    unique_words = set()

    # Pour chaque phrase, ajouter les mots uniques au set
    for phrases in post:
        vocab_complexity = {}
        for phrase in phrases.split():
            words = word_tokenize(phrase)
            for word in words:
                unique_words.add(word)

        # Compter le nombre total de mots
        total_word_count = 0
        for phrase in phrases:
            total_word_count += len(word_tokenize(phrase))

        # Calculer la complexité du vocabulaire en divisant le nombre de mots uniques par le nombre total de mots
        vocab_complexity["complexity"] = len(unique_words) / total_word_count
        liste_finale.append(vocab_complexity)
    return liste_finale
